In [1]:
!git clone https://github.com/mistralai/mistral-finetune.git

Cloning into 'mistral-finetune'...


In [2]:
%cd mistral-finetune/

/teamspace/studios/this_studio/mistral-finetune


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!pip install -r requirements.txt

In [5]:
!pip install huggingface_hub

In [1]:
# Alternatively, you can download the model from mistral

!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2025-01-08 11:06:43--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.7.117, 172.67.70.68, 104.26.6.117, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.7.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar.1’

mistral-7B-v0.3.tar   0%[                    ] 132.13M  27.9MB/s    eta 8m 55s ^C


In [2]:
!DIR=~/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [6]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /teamspace/studios/this_studio/mistral-finetune/data/mistral.train.jsonl

In [7]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating /teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl ...

100%|███████████████████████████████████| 29999/29999 [00:03<00:00, 8164.35it/s]
1it [00:03,  3.73s/it]
No errors! Data is correctly formatted!
Stats for /teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:22:54",
        "data_tokens": 2046750,
        "train_tokens": 49152000,
        "epochs": "24.01",
        "max_steps": 750,
        "data_tokens_per_dataset": {
            "/teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl": "2046750.0"
        },
        "train_tokens_per_dataset": {
            "/teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl": "49152000.0"
        },
        "epochs_per_dataset": {
            "/teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl": "24.0"
        }


In [9]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/teamspace/studios/this_studio/mistral-finetune/data/mistral_train_dataset.jsonl"
  data: "" 
  eval_instruct_data: "/teamspace/studios/this_studio/mistral-finetune/data/mistral_val_dataset.jsonl"

# model
model_id_or_path: "/teamspace/studios/this_studio/mistral_models"
lora:
  rank: 64  # Keep low-rank adapters to fit the model on available memory

# optim
seq_len: 2048  # Max sequence length; adjust if you face memory constraints
batch_size: 2  # Increase to 4 if memory permits for faster training
max_steps: 500  # Slightly increase for sufficient epochs over 30k dataset
optim:
  lr: 4e-5  # Lower learning rate for better stability with fine-tuning
  weight_decay: 0.05  # Slightly lower to avoid over-regularization
  pct_start: 0.1  # Adjust warm-up period for smoother LR schedule

# other
seed: 42  # Reproducibility
log_freq: 10  # Log every 10 steps
eval_freq: 50  # Evaluate more frequently for smaller datasets
no_eval: False
ckpt_freq: 100  # Save checkpoints frequently to avoid data loss

save_adapters: True  # Save LoRA adapters only; save full model if needed by setting to False

run_dir: "/teamspace/studios/this_studio/output_lora"

wandb:
  project: "Mistral-7B-v3"
  run_name: "fine-tuning-mistral"
  key: "057a18c9b0ad17d5aa800a60fc6907dfb87c226e"
  offline: False

"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [1]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

%cd mistral-finetune/

/teamspace/studios/this_studio/mistral-finetune


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# start training
!torchrun --nproc-per-node 1 -m train example.yaml

args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/teamspace/studios/this_studio/mistral-finetune/data/mistral.train.jsonl', eval_instruct_data='/teamspace/studios/this_studio/mistral-finetune/data/mistral.val.jsonl', instruct=InstructArgs(shuffle=True, dynamic_chunk_fn_call=True)), model_id_or_path='/teamspace/studios/this_studio/mistral_models', run_dir='/teamspace/studios/this_studio/output_mixed_data_v2', optim=OptimArgs(lr=2e-05, weight_decay=0.05, pct_start=0.1), seed=0, num_microbatches=1, seq_len=2048, batch_size=4, max_norm=1.0, max_steps=300, log_freq=10, ckpt_freq=60, save_adapters=False, no_ckpt=False, num_ckpt_keep=3, eval_freq=50, no_eval=False, checkpoint=True, world_size=1, wandb=WandbArgs(project='Mistral-7B-v3', offline=False, key='057a18c9b0ad17d5aa800a60fc6907dfb87c226e', run_name='fine-tuning-mistral'), mlflow=MLFlowArgs(tracking_uri=None, experiment_name=None), lora=LoraArgs(enable=True, rank=32, dropout=0.0, scaling=2.0))
2025-01-09 07:05:50 (U

In [3]:
!pwd

/teamspace/studios/this_studio


In [2]:
!python mistral-finetune/evaluate.py \
    --model_path /teamspace/studios/this_studio/mistral_models \
    --tokenizer_path /teamspace/studios/this_studio/mistral_models/tokenizer.model.v3 \
    --test_file /teamspace/studios/this_studio/mistral-finetune/data/test.model-agnostic.json \
    --output_file base_mistral_agnostic.json


[INFO] Loading Mistral model from /teamspace/studios/this_studio/mistral_models
[INFO] Loading tokenizer from /teamspace/studios/this_studio/mistral_models/tokenizer.model.v3
[INFO] Reading test data from /teamspace/studios/this_studio/mistral-finetune/data/test.model-agnostic.json
[INFO] Starting inference loop ...
Processing Samples: 100%|███████████████| 1500/1500 [08:52<00:00,  2.81sample/s]
[INFO] Writing predictions to base_mistral_agnostic.json
[INFO] Done! Inference complete.


In [2]:
!pkill -f python

: 

In [1]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", local_dir=mistral_models_path)


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/teamspace/studios/this_studio/mistral_models/7B-v0.3'